In [ ]:
lr = LinearRegression()
''mpg', y = ''horsdee
lr.fit(csv['mpg'],csv['horsepower'])

lr 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

csv = pd.read_csv('auto-mpg.csv')
csv